In [ ]:
!./build.sh
# !./sin_perf > sin_data.txt
!./sin_perf_pure_vec > sin_pure_data_vec.txt

In [ ]:
%pylab inline

In [ ]:
#D = loadtxt("sin_data.txt")
#x = D[:,0]
#sin_gf = D[:,2]
D = loadtxt("sin_pure_data_vec.txt")
x2 = D[:,0]
sin_pure = D[:,2]

GHz = 1e9
KB = 1024
MB = 1024**2

#cpu_freq = 2.4 * GHz
cpu_freq = 5.2 * GHz * 1.2
L1 = 512 * KB
L2 = 2048 * KB
L3 = 16 * MB
k = 8 * 2 # 8 bytes per element, 2 arrays

figure(figsize=(20, 12))
#loglog(x, sin_gf, ".", label="GFortran Intrinsic")
semilogx(x2, sin_pure * cpu_freq, ".", label="Pure double double")
semilogx([L1/k, L1/k], [0, 2], "-", label="L1")
semilogx([L2/k, L2/k], [0, 2], "-", label="L2")
semilogx([L3/k, L3/k], [0, 2], "-", label="L3")
legend()
xlabel("Array length [double]")
ylabel("Time of sin(x) per array element")
grid()
xlim([1e2, None])
ylim([0, 2])
savefig("perf1.pdf")
show()